# AI_MHW_images Round 2

In [13]:
# imports
import os
import numpy as np

import pandas

from mhw_analysis.ai import images as ai_images

# Run on Intermediate MHW Systems

## Load intermediate table

In [4]:
path = '/home/xavier/Projects/Oceanography/AI/MHWS'

In [5]:
mhw_systems = pandas.read_hdf(os.path.join(path, 'MHW_sys_intermediate.hdf'))

In [7]:
mhw_systems.head()

,NSpax,category,mask_Id,max_area,xcen,xboxmin,xboxmax,ycen,yboxmin,yboxmax,zcen,zboxmin,zboxmax,date,lat,lon,max_time,null_time
Id,,,,,,,,,,,,,,,,,,
4768,238157,1,24434,2530,146.690933,64,230,716.595215,621,827,6875.685059,6656,7068,2000-10-28,-53.202267,179.273804,730521,729790
6943,48894,1,35546,1700,100.062790,70,130,729.652039,649,809,3124.972656,3056,3211,1990-07-22,-64.859303,182.538010,726660,727025
7526,151730,4,38982,4832,100.569588,71,141,1425.960693,0,1439,4660.063965,4600,4747,1994-10-05,-64.732603,356.615173,728189,728554
8828,72612,4,46816,2046,107.257065,76,154,303.012939,230,387,8077.781738,7980,8132,2004-02-12,-63.060734,75.878235,731627,732723
9588,125047,2,50773,2424,103.733215,77,156,1380.768066,0,1439,13601.375977,13551,13668,2019-03-29,-63.941696,345.317017,737140,735679


## Grab Z500

In [11]:
# Example near lon=0
idx = np.argmin(mhw_systems.lon)
mhw_systems.iloc[idx]

NSpax             48456
category              2
mask_Id          729524
max_area           1284
xcen            237.687
xboxmin             202
xboxmax             284
ycen          0.0939331
yboxmin               0
yboxmax            1439
zcen            1527.32
zboxmin            1489
zboxmax            1606
date         1986-03-08
lat            -30.4532
lon            0.148483
max_time         725061
null_time        724696
Name: 202585, dtype: object

### Without climate subtraction